#  Generative Adversarial Networks

**"Deep neural net architectures comprised of two nets, pitting one against the other"**



## Pregled
* GAN

## GAN
- Jedna od najinovativnijih i najpopularnijih novih ideja u ML u poslednjih 10 godina
- Generativni model
- Svakim danom sve više radova https://imgur.com/IFXxquc
- I dalje ne znamo baš kako funkcioniše
- Dve mreže koje se "bore": generator i diskriminator
- Treniramo ih naizmenično u isto vreme, nemamo fiksnu funkciju troška
- Vrlo se teško treniraju i lako divergiraju

## Struktura
- "Prevarant" i "policajac": prevarant uči sve bolje da pravi lažne novčanice, policajac uči sve bolje da ih razlikuje
  - Rezultat: prevarant postane jako dobar
- Dakle, generator treba da generiše primere takve da diskriminator ne može da ih razlikuje od pravih
- Trening režim: https://imgur.com/807GTFL
- Generator menja svoje težine tako da dobije što veći skor od diskriminatora
- Diskriminator menja svoje težine tako da da što manji skor generatoru ali što veći skor realnim primerima
- Funkcija troška je mreža!
-Trik: treba paziti da diskriminator ne ode previše ispred generatora, ne treniramo ga do konvergencije
  - Ovaj i još mnogo trikova
  - WGAN, DCGAN, ...

- Istorija ML: 
  - Feature engineering pre modela
  - Neuralne mreže (nema potrebe za feature engineeringom ali nam treba cost funkcija)
  - GAN (nema potrebe za cost funkcijom ali moramo da znamo kako da radimo optimizaciju)
  - ??? (learning to learn)
  
## Rezultati
- Aritmetika u latentnom prostoru
- Interpolacija između slika
- SuperResolution
- Interaktivni GAN
- Style transfer

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("MNIST_data")

def generator(z,reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        hidden1=tf.layers.dense(inputs=z,units=128,activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1,units=128,activation=tf.nn.leaky_relu)
        output=tf.layers.dense(inputs=hidden2,units=784,activation=tf.nn.tanh)
        
        return output
    
def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        hidden1=tf.layers.dense(inputs=X,units=128,activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1,units=128,activation=tf.nn.leaky_relu)
        logits=tf.layers.dense(hidden2,units=1)
        output=tf.sigmoid(logits)
        
        return output,logits
    

tf.reset_default_graph()

real_images=tf.placeholder(tf.float32,shape=[None,784])
z=tf.placeholder(tf.float32,shape=[None,100])

G=generator(z)
D_output_real,D_logits_real=discriminator(real_images)
D_output_fake,D_logits_fake=discriminator(G,reuse=True)

def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

D_real_loss=loss_func(D_logits_real,tf.ones_like(D_logits_real)*0.9) #Smoothing for generalization
D_fake_loss=loss_func(D_logits_fake,tf.zeros_like(D_logits_real))
D_loss=D_real_loss+D_fake_loss

G_loss= loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

lr=0.001

#Do this when multiple networks interact with each other
tvars=tf.trainable_variables()  #returns all variables created(the two variable scopes) and makes trainable true
d_vars=[var for var in tvars if 'dis' in var.name]
g_vars=[var for var in tvars if 'gen' in var.name]

D_trainer=tf.train.AdamOptimizer(lr).minimize(D_loss,var_list=d_vars)
G_trainer=tf.train.AdamOptimizer(lr).minimize(G_loss,var_list=g_vars)

batch_size=100
epochs=100
init=tf.global_variables_initializer()

samples=[] #generator examples

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        num_batches=mnist.train.num_examples//batch_size
        for i in range(num_batches):
            batch=mnist.train.next_batch(batch_size)
            batch_images=batch[0].reshape((batch_size,784))
            batch_images=batch_images*2-1
            batch_z=np.random.uniform(-1,1,size=(batch_size,100))
            _=sess.run(D_trainer,feed_dict={real_images:batch_images,z:batch_z})
            _=sess.run(G_trainer,feed_dict={z:batch_z})
            
        print("on epoch{}".format(epoch))
        
        sample_z=np.random.uniform(-1,1,size=(1,100))
        gen_sample=sess.run(generator(z,reuse=True),feed_dict={z:sample_z})
        
        samples.append(gen_sample)

plt.imshow(samples[0].reshape(28,28))

plt.imshow(samples[99].reshape(28,28))


## Resursi
[Primene](https://medium.com/@jonathan_hui/gan-some-cool-applications-of-gans-4c9ecca35900)
